# Blogger Word Cloud Generator 📝☁️

ブログのURLと分析期間を入力して、下の▶ボタンを押してください。

**注:** 除外したい単語は、GitHubリポジトリにある `stopwords.txt` ファイルを編集してください。

In [ ]:
#@title ◆ 設定と実行 ◆
#@markdown --- 
#@markdown ### 1. あなたのブログとリポジトリの情報を入力
blog_url = "https://amufaamo.blogspot.com/" #@param {type:"string"}
#@markdown GitHubリポジトリのURL (`https://github.com/ユーザー名/リポジトリ名`)
github_repo_url = "https://github.com/amufaamo/blog-to-tagcloud" #@param {type:"string"}
days_to_analyze = 30 #@param {type:"slider", min:7, max:1000, step:1}

#@markdown --- 
#@markdown **設定が終わったら、このセルを実行してください（左の▶ボタンをクリック）**
import os

print("環境の準備中です...")
print("1. 必要なライブラリをインストールしています。")
!pip install requests beautifulsoup4 janome wordcloud matplotlib > /dev/null 2>&1

print("2. 日本語フォントをダウンロードしています。")
font_path = '/content/NotoSansCJKjp-Regular.otf'
if not os.path.exists(font_path):
    !wget -q -O {font_path} https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/Japanese/NotoSansCJKjp-Regular.otf

print("3. GitHubから除外ワードリスト(stopwords.txt)を取得しています。")
# GitHubのURLをRAWコンテンツのURLに変換
stopwords_url = github_repo_url.replace('github.com', 'raw.githubusercontent.com') + '/main/stopwords.txt'
!wget -q -O /content/stopwords.txt {stopwords_url}

print("準備が完了しました！\n")

# --- 以下、分析用のコード ---
import requests
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
from collections import Counter
import re
from datetime import datetime, timedelta, timezone
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def load_stopwords(filepath='/content/stopwords.txt'):
    """ファイルから除外ワードを読み込み、セットとして返す"""
    if not os.path.exists(filepath):
        return set()
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = {line.strip() for line in f if line.strip()}
    print(f"{len(stopwords)}個の除外ワードを読み込みました。")
    return stopwords

def analyze_blog(base_url, days_to_check, stopwords):
    all_text = ""
    jst = timezone(timedelta(hours=9))
    start_date = datetime.now(jst) - timedelta(days=days_to_check)
    current_url = base_url
    page_num = 1
    print(f"ブログの分析を開始します。対象期間: {days_to_check}日間")
    while current_url:
        print(f"ページ {page_num} を取得中...")
        try:
            response = requests.get(current_url)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
        except:
            print("エラー: ページにアクセスできませんでした。")
            break
        posts = soup.find_all('article', class_='post-outer-container')
        if not posts and page_num == 1: return None
        page_contains_valid_posts = False
        for post in posts:
            time_tag = post.find('time', class_='published')
            body_tag = post.find('div', class_='post-body')
            if not time_tag or not body_tag: continue
            post_date = datetime.fromisoformat(time_tag['datetime'])
            if post_date >= start_date:
                page_contains_valid_posts = True
                all_text += body_tag.get_text() + "\n"
        if not page_contains_valid_posts and page_num > 1: break
        older_posts_link = soup.find('a', class_='blog-pager-older-link')
        if older_posts_link and older_posts_link.has_attr('href'):
            current_url = older_posts_link['href']
            page_num += 1
        else:
            current_url = None
    if not all_text: return None
    print("\nテキストの解析中...")
    t = Tokenizer()
    words = [token.surface for token in t.tokenize(all_text) 
             if token.surface not in stopwords and 
             token.part_of_speech.startswith(('名詞', '動詞', '形容詞')) and 
             len(token.surface) > 1 and not re.match(r'^[0-9a-zA-Z]+$', token.surface)]
    return Counter(words)

# メイン処理
custom_stopwords = load_stopwords()
word_counter = analyze_blog(blog_url, days_to_analyze, custom_stopwords)

if word_counter:
    print("\n--- ✅単語の頻度ランキング TOP 50 ---")
    for word, count in word_counter.most_common(50):
        print(f"{word}: {count}回")
    
    print("\nワードクラウド画像を生成中...")
    if os.path.exists(font_path) and os.path.getsize(font_path) > 1000000:
      try:
        wordcloud = WordCloud(
            width=1200, height=600, background_color='white',
            font_path=font_path, max_words=150, colormap='viridis'
        ).generate_from_frequencies(word_counter)
        
        plt.figure(figsize=(15, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
      except OSError as e:
        print(f"\nエラー: ワードクラウドの生成中にエラーが発生しました。フォントファイルが破損している可能性があります。")
        print(f"エラー詳細: {e}")
    else:
        print(f"エラー：フォントファイルが正しくダウンロードできませんでした。もう一度セルを実行するか、「ランタイムの再起動」を試してください。")
else:
    print("エラー: 分析対象の記事が見つかりませんでした。URLや期間を確認してください。")